# Петров Сергей

# Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import math

from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [2]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [3]:
r_state = 42

# Подготовка датасетов

In [128]:
adult_df = pd.read_csv('adult.csv')

In [129]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [130]:
adult_df['capital.gain'] = (adult_df['capital.gain'] != 0).astype('int')
adult_df['capital.loss'] = (adult_df['capital.loss'] != 0).astype('int')

In [131]:
num_cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

In [132]:
col_to_cat = ['age', 'fnlwgt', 'education.num', 'hours.per.week']
for col in col_to_cat:
    _, bins = pd.qcut(adult_df[col], q=10, duplicates='drop', retbins=True)
    
    bins[0] = -np.inf
    bins[-1] = np.inf
    adult_df[col + '_cat'] = pd.cut(adult_df[col], bins=bins)

In [133]:
adult_df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [134]:
adult_df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,age_cat,fnlwgt_cat,education.num_cat,hours.per.week_cat
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,1,40,United-States,<=50K,"(58.0, inf]","(65716.0, 106648.0]","(7.0, 9.0]","(35.0, 40.0]"
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,1,18,United-States,<=50K,"(58.0, inf]","(130856.0, 158662.0]","(7.0, 9.0]","(-inf, 24.0]"
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,1,40,United-States,<=50K,"(58.0, inf]","(178356.0, 196338.0]","(9.0, 10.0]","(35.0, 40.0]"
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,1,40,United-States,<=50K,"(50.0, 58.0]","(130856.0, 158662.0]","(-inf, 7.0]","(35.0, 40.0]"
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,1,40,United-States,<=50K,"(37.0, 41.0]","(259873.0, 329054.0]","(9.0, 10.0]","(35.0, 40.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K,"(-inf, 22.0]","(259873.0, 329054.0]","(9.0, 10.0]","(35.0, 40.0]"
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,"(26.0, 30.0]","(219632.0, 259873.0]","(11.0, 13.0]","(35.0, 40.0]"
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,"(37.0, 41.0]","(130856.0, 158662.0]","(7.0, 9.0]","(35.0, 40.0]"
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,"(50.0, 58.0]","(130856.0, 158662.0]","(7.0, 9.0]","(35.0, 40.0]"


In [135]:
adult_train_df, adult_test_df = train_test_split(adult_df,  test_size=0.25, random_state=r_state)

In [136]:
adult_cat_col = adult_df.select_dtypes(include=object).columns.to_list()
adult_cat_col.remove('income')
print(adult_cat_col)

['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']


In [137]:
adoult_target_mask = adult_df.columns.to_list()
adoult_target_mask.remove('income')
adoult_target_mask

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country',
 'age_cat',
 'fnlwgt_cat',
 'education.num_cat',
 'hours.per.week_cat']

In [138]:
from sklearn.datasets import load_iris

In [139]:
iris_data = load_iris(as_frame=True)

In [140]:
iris_df = iris_data['data']
iris_df['target'] = iris_data['target']

In [141]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [142]:
train_df, test_df = train_test_split(iris_df, test_size=0.5, random_state=42)

In [143]:
x_cols = iris_df.columns.to_list()
x_cols.remove('target')

In [144]:
x_cols

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [145]:
X_train = train_df[x_cols]
y_train = train_df['target']

X_test = test_df[x_cols]
y_test = test_df['target']

# Наивный Байесовский классификатор для непрервыных фичей(гауссиан)

In [214]:
def gausFunc(x, m, d):
    # return math.exp(-(x - m)**2/(2*d**2))/math.sqrt(2*math.pi*d**2)
    return np.exp(-(x - m) ** 2 / (2 * d ** 2)) / np.sqrt(2 * np.pi * d ** 2)


class MyGaussianNB:
    def __init__(self):
        
        # Массив средних значений для каждой фичи, для каждого класса
        # (размер NxK, где N - кол-во фичей, K - кол-во классов)
        self.mus = {}

        # Массив стандартных среднеквадратических отклоенний для каждой фичи
        # для каждого класса (размер NxK, где N - кол-во фичей, K - кол-во классов)
        self.sigmas = {}
        
        self.p = {}
        
        self.coluns = []
        self.costil = {}
    
    def fit(self, X, y):
        df = X.copy()
        df['target'] = y
        
        self.columns = y.unique()
        grouped = df.groupby(['target'])
        
        for u in self.columns:
            group = grouped.get_group(u).select_dtypes(include=np.number)
            nu = str(u)
            self.costil[nu] = u
            
            self.mus[nu] = group.mean()
            self.sigmas[nu] = group.std()
            self.p[nu] = group.shape[0]/X.shape[0]
        
        
    def predict(self, X):
        return self.likelihood(X).idxmax(axis=1).apply(lambda x : self.costil[x])
    
    def predict_proba(self, X):
        # Применить функцию softmax (scipy.special.softmax) на выходы likelihood
        return softmax(self.likelihood(X), axis = -1)
    
    def likelihood(self, X):
        # Для каждого класса посчитать функции правдоподобия для каждой фичи
        # Прологарифмировать и сложить их между собой и добавить логарифм
        # априорной вероятности соответствующего класса
        # Получится матрица размером NxK, где N - кол-во строк, K - кол-во классов
        df = X.select_dtypes(include=np.number)
        categ = [* self.mus]
        features = df.columns
        
        for cat in categ:
            df[cat] = sum([np.log(gausFunc(df[feat],self.mus[cat][feat], self.sigmas[cat][feat] ))for feat in features]) + np.log(self.p[cat])
            #print([np.log(gausFunc(df[feat],self.mus[cat][feat], self.sigmas[cat][feat] ))for feat in features])
        return df[categ]

In [215]:
gnb = MyGaussianNB()

In [216]:
gnb.fit(X_train, y_train)

In [217]:
gnb.likelihood(X_test)

,1,2,0
73,-1.327471,-6.637869,-256.689725
18,-31.871922,-60.222791,-3.479330
118,-28.668378,-9.290720,-798.078356
78,-1.255465,-5.454553,-256.506249
76,-2.492752,-4.595382,-294.339389
...,...,...,...
113,-7.331660,-3.985124,-382.903750
33,-43.859258,-74.085996,-2.661841
138,-4.187132,-2.876021,-328.702924
101,-5.702914,-2.638833,-383.524409


In [220]:
iris_gnb = gnb.predict(X_test)

73     1
18     0
118    2
78     1
76     1
      ..
113    2
33     0
138    2
101    2
62     1
Length: 75, dtype: int64

In [219]:
gnb.predict_proba(X_test)

,1,2,0
73,9.950843e-01,4.915673e-03,1.245781e-111
18,4.669321e-13,2.273150e-25,1.000000e+00
118,3.840521e-09,1.000000e+00,0.000000e+00
78,9.852127e-01,1.478732e-02,1.378870e-111
76,8.911586e-01,1.088414e-01,1.594303e-127
...,...,...,...
113,3.400876e-02,9.659912e-01,2.646700e-165
33,1.282889e-18,9.569373e-32,1.000000e+00
138,2.123011e-01,7.876989e-01,2.463395e-142
101,4.461346e-02,9.553865e-01,3.661639e-166


In [87]:
gnb.predict_proba(X_test).sum()

1    1.0
2    1.0
0    1.0
dtype: float64

In [88]:
np.unique(iris_gnb == y_test, return_counts = True)

(array([False,  True]), array([ 1, 74]))

In [89]:
np.unique(gnb.predict(X_train) == y_train, return_counts = True)

(array([False,  True]), array([ 6, 69]))

In [90]:
accuracy_score(y_train, gnb.predict(X_train)), accuracy_score(y_test, gnb.predict(X_test))

(0.92, 0.9866666666666667)

# Наивный баесовский классификатор для категориальных фичей

In [221]:
class MyCategoryNB:
    def __init__(self, alpha=1):
        # Сглаживающая константа, для того, чтобы предотвратить перемножение на 0
        # Если какой-то категории не встречается
        self.alpha = alpha
        self.p = []
        
        self.col = []
        self.cat = []
        self.uniq = []
        self.ap = []
    
    def fit(self, X, y):
        df = X.copy()
        
        self.col = df.columns
        
        self.uniq = [df[c].unique().tolist() for c in self.col]
        
        self.cat = y.unique()
        
        df['target'] = y
        
        grouped = df.groupby(['target'])
        
        for cat in self.cat:
            group = grouped.get_group(cat)
            self.ap += [group.shape[0]/df.shape[0]]
            self.p += [[[(group[(group[self.col[c]] == un)].shape[0] + self.alpha)/ (group.shape[0] + self.alpha * len(self.uniq[c])) for un in self.uniq[c]] for c in range(len(self.col))]]
        
    def predict(self, X):
        
        # Используя метод likelihood посчитать функции правдоподобия для каждой строчки для каждого класса
        # Для каждой строчки выбрать класс с максимальным значением функции правдоподобия
        return self.likelihood(X).idxmax(axis=1)
    
    def predict_proba(self, X):
        # Применить функцию softmax (scipy.special.softmax) на выходы likelihood
        return softmax(self.likelihood(X), axis = -1)
    
    def likelihood(self, X):
        # Для каждого класса посчитать функции правдоподобия для каждой фичи
        # Прологарифмировать и сложить их между собой и добавить логарифм априорной вероятности соответствующего класса
        # Получится матрица размером NxK, где N - кол-во строк, K - кол-во классов
        df = X.copy()
        for cat_ind in range(len(self.cat)):
            
            df[str(self.cat[cat_ind])] = sum([df[self.col[col_ind]].apply(lambda x: np.log(self.p[cat_ind][col_ind][self.uniq[col_ind].index(x)])) for col_ind in range(len(self.col))]) + np.log(self.ap[cat_ind])
            
        return df[[str(cat) for cat in self.cat]]

In [222]:
catNB = MyCategoryNB()

In [223]:
catNB.fit(adult_train_df[adult_cat_col], adult_train_df['income'])#adult_cat_col

In [224]:
catNB.ap

[0.24148239148239148, 0.7585176085176085]

In [225]:
catNB.likelihood(adult_test_df[adult_cat_col])

,>50K,<=50K
14160,-8.985511,-8.105710
27048,-13.658819,-7.550948
28868,-11.562482,-11.409089
5667,-14.259245,-8.829219
7827,-19.952806,-11.670561
...,...,...
26658,-12.802226,-8.969198
29401,-14.864252,-8.496472
27265,-6.512580,-7.378402
30339,-14.755816,-8.163259


In [226]:
catNB.predict(adult_test_df[adult_cat_col])

14160    <=50K
27048    <=50K
28868    <=50K
5667     <=50K
7827     <=50K
         ...  
26658    <=50K
29401    <=50K
27265     >50K
30339    <=50K
8610     <=50K
Length: 8141, dtype: object

In [227]:
catNB.predict_proba(adult_test_df[adult_cat_col])

,>50K,<=50K
14160,0.293219,0.706781
27048,0.002220,0.997780
28868,0.461727,0.538273
5667,0.004364,0.995636
7827,0.000253,0.999747
...,...,...
26658,0.021185,0.978815
29401,0.001713,0.998287
27265,0.703876,0.296124
30339,0.001369,0.998631


In [228]:
accuracy_score(adult_train_df['income'], catNB.predict(adult_train_df[adult_cat_col])), accuracy_score(adult_test_df['income'], catNB.predict(adult_test_df[adult_cat_col]))

(0.7947174447174448, 0.7914261147279203)

# Совмещение Баейсов

In [230]:
class MySemiNB:
    def __init__(self, alpha=1):
        self.gaus = MyGaussianNB()
        self.cat = MyCategoryNB()
        self.ap = []
        
    def fit(self, X, y):
        self.gaus.fit(X.select_dtypes(include=np.number), y)
        self.cat.fit(X.select_dtypes(include=object), y)
        self.ap = np.log(self.cat.ap[:])
        
    def predict(self, X):
        
        return self.likelihood(X).idxmax(axis=1)
    
    def predict_proba(self, X):
        # Применить функцию softmax (scipy.special.softmax) на выходы likelihood
        return softmax(self.likelihood(X), axis = -1)
    
    def likelihood(self, X):
        return self.gaus.likelihood(X.select_dtypes(include=np.number)) + self.cat.likelihood(X.select_dtypes(include=object)) - self.ap

In [231]:
msnb = MySemiNB()

In [232]:
msnb.fit(adult_train_df[adoult_target_mask], adult_train_df['income'] )

In [233]:
msnb.likelihood(adult_train_df[adoult_target_mask])

,>50K,<=50K
29,-49.700041,-54.584966
12181,-28.271460,-29.426327
18114,-35.052833,-32.578553
4278,-34.195667,-34.575340
12050,-39.289631,-30.415795
...,...,...
29802,-38.951484,-29.323403
5390,-28.457429,-29.812428
860,-36.755975,-48.510592
15795,-28.416502,-27.463212


In [234]:
msnb.predict(adult_test_df[adoult_target_mask])

14160    <=50K
27048    <=50K
28868    <=50K
5667     <=50K
7827     <=50K
         ...  
26658    <=50K
29401    <=50K
27265    <=50K
30339    <=50K
8610     <=50K
Length: 8141, dtype: object

In [235]:
catNB.predict_proba(adult_test_df[adoult_target_mask])

,>50K,<=50K
14160,0.293219,0.706781
27048,0.002220,0.997780
28868,0.461727,0.538273
5667,0.004364,0.995636
7827,0.000253,0.999747
...,...,...
26658,0.021185,0.978815
29401,0.001713,0.998287
27265,0.703876,0.296124
30339,0.001369,0.998631


In [236]:
accuracy_score(adult_train_df['income'], msnb.predict(adult_train_df[adoult_target_mask])), accuracy_score(adult_test_df['income'], msnb.predict(adult_test_df[adoult_target_mask]))

(0.8224406224406224, 0.8135364205871515)